In [14]:
import shap
import pandas as pd
import pickle
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from alibi.explainers import AnchorTabular
import time

In [27]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_full = pd.read_csv("./data/base/train-white.csv", sep=",")
test_full = pd.read_csv("./data/base/test-white.csv", sep=",")

In [3]:
target = "quality"
model_name = "rf"

In [4]:
with open('./model/selected-{}.pickle'.format(model_name), 'rb') as f:
    model = pickle.load(f)

In [5]:
X_train = copy.deepcopy(train_full.drop(columns=target))
X_test = copy.deepcopy(test_full.drop(columns=target))
y_train = train_full[target]
y_test = test_full[target]

In [7]:
predict_fn = lambda x: model.predict_proba(x)
feature_names = X_train.columns.to_list()
explainer = AnchorTabular(predict_fn, feature_names)
explainer.fit(X_train.to_numpy())

X does not have valid feature names, but StandardScaler was fitted with feature names


AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': None, 'disc_perc': (25, 50, 75)},
  'version': '0.9.2'}
)

In [17]:
X_test_np = X_test.to_numpy()
test_length = X_test_np.shape[0]

In [15]:
def time_convert(sec):
  mins = sec // 60
  sec = sec % 60
  hours = mins // 60
  mins = mins % 60
  return "Time Lapsed = {0}:{1}:{2}".format(int(hours),int(mins),sec)

In [24]:
my_df = pd.DataFrame(X_test_np[0]).transpose()
my_df.columns = feature_names
my_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,6.8,0.67,0.3,13.0,0.29,22.0,193.0,0.9984,3.08,0.67,9.0


In [ ]:
X_test_np = X_test.to_numpy()
X_test_np.shape
start_time = time.time()

anchors = []
for i, dp in enumerate(X_test_np):
    anchors.append(explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, test_length, time_convert(time_lapsed)), end="\r", flush=True)



In [49]:
anchors[0]["data"]

{'anchor': ['alcohol <= 9.47',
  'volatile acidity > 0.33',
  'density > 1.00',
  'pH <= 3.09',
  'free sulfur dioxide <= 23.00'],
 'precision': 0.9572649572649573,
 'coverage': 0.0029,
 'raw': {'feature': [10, 1, 7, 8, 5],
  'mean': [0.4316630922207058,
   0.6234676007005254,
   0.785472972972973,
   0.9076655052264808,
   0.9572649572649573],
  'precision': [0.4316630922207058,
   0.6234676007005254,
   0.785472972972973,
   0.9076655052264808,
   0.9572649572649573],
  'coverage': [0.2496, 0.0583, 0.0377, 0.0154, 0.0029],
  'examples': [{'covered_true': array([[7.0000e+00, 3.9000e-01, 3.1000e-01, 5.3000e+00, 1.6900e-01,
            3.2000e+01, 1.6200e+02, 9.9650e-01, 3.2000e+00, 4.8000e-01,
            8.8000e+00],
           [6.9000e+00, 3.9000e-01, 2.2000e-01, 4.3000e+00, 3.0000e-02,
            1.0000e+01, 1.0200e+02, 9.9300e-01, 3.0000e+00, 8.7000e-01,
            9.2000e+00],
           [6.6000e+00, 2.5000e-01, 3.9000e-01, 1.4500e+00, 4.0000e-02,
            4.0000e+01, 8.9000e

In [32]:
from pymongo import MongoClient

In [38]:
# Create a MongoClient to the running mongod instance
client = MongoClient("localhost", 27017)

# Get a reference to a particular database
db = client["pipeline"]
model_collection = db["model"]
anchor_meta_collection = db["anchor_meta"]
anchor_collection = db["anchor"]

stage = "selection"
model_name = model_name

In [39]:
model_id = model_collection.find_one(
            {"stage": stage, "model_name": model_name}
        )["_id"]

In [50]:
anchor_meta_collection.insert_one(
    {
        "model_id": model_id,
        "info": anchors[0]["meta"],
        "classes": model[model_name].classes_.tolist(),
        "features": X_train.columns.tolist()
    }
)

In [51]:
anchor_meta_id = anchor_meta_collection.find_one(
            {"model_id": model_id}
        )["_id"]

In [52]:
for index, elem in enumerate(anchors):
    anchor_collection.insert_one(
        {
            "anchor_meta_id": anchor_meta_id,
            "index": index,
            'anchor': elem["data"]['anchor'],
            'precision': elem["data"]['precision'],
            'coverage': elem["data"]['coverage']
        }
    )